In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
DATA_DIR = "/user/projects/project-3-ss3382/medicare_data_csv"

In [3]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [4]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [5]:
cursor = connection.cursor()

In [6]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

### The plan is to build the providers table which is all Dermatology providers in 2022 that got payments from Medicare system 

In [7]:
connection.rollback()

query = """

drop table if exists providers CASCADE;

"""

cursor.execute(query)

connection.commit()

In [8]:
connection.rollback()


query = """

create table providers (
  provider_npi varchar(32),
  provider_last_name varchar(32),
  provider_first_name varchar(32),
  provider_credentials varchar(32),
  provider_gender varchar(2),
  provider_state varchar(2),
  provider_zip varchar(32),
  primary key (provider_npi)
);

"""

cursor.execute(query)

connection.commit()

In [9]:
connection.rollback()

query = f"""

copy providers
from '{DATA_DIR}/providers.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [10]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from providers
where provider_state = 'FL'

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,provider_npi,provider_last_name,provider_first_name,provider_credentials,provider_gender,provider_state,provider_zip
0,1003038555,Pascoe,David,M.D.,M,FL,32308
1,1003077652,Patel,Utpalkumar,M.D. PH.D.,M,FL,34601
2,1003083247,Lombardi,Milan,MD,M,FL,33713
3,1003142084,Singer,Thomas,DO,M,FL,33907
4,1003811548,Teltscher,Janet,MD,F,FL,33180
...,...,...,...,...,...,...,...
989,1992728802,Dewberry,Charles,DO,M,FL,34450
990,1992837900,Sherman,Ronald,DO,M,FL,32174
991,1992894414,Crater,Scott,MD,M,FL,33919
992,1992897938,Weinkle,Susan,M.D.,F,FL,34209


## Building Drug Table


In [11]:
connection.rollback()

query = """

drop table if exists drugs;

"""

cursor.execute(query)

connection.commit()

In [12]:
connection.rollback()


query = """

create table drugs (
  drug_code varchar(32),
  drug_description text,
  if_drugg boolean,
  primary key (drug_code)
);

"""

cursor.execute(query)

connection.commit()

In [13]:
connection.rollback()

query = f"""

copy drugs
from '{DATA_DIR}/drugs.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [14]:
rollback_before_flag = True
rollback_after_flag = True

query = """

with report as (
select 
r.provider_npi,
r.code,
r.total_amount_paid as weight
from services_rendered r
     inner join (
         select provider_npi
            from providers
            where provider_state = 'FL') k on k.provider_npi = r.provider_npi
)

select distinct d.drug_code, d.drug_description, r.provider_npi
from report r
    inner join
    drugs d on r.code = d.drug_code

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,drug_code,drug_description,provider_npi
0,J3245,"Injection, tildrakizumab, 1 mg",1265495782
1,J7308,Aminolevulinic acid hcl for topical administra...,1215979398
2,J3301,"Injection, triamcinolone acetonide, not otherw...",1003844937
3,J3301,"Injection, triamcinolone acetonide, not otherw...",1609918671
4,J3301,"Injection, triamcinolone acetonide, not otherw...",1659358521
...,...,...,...
677,J3301,"Injection, triamcinolone acetonide, not otherw...",1114914934
678,J9250,"Methotrexate sodium, 5 mg",1407820764
679,J3301,"Injection, triamcinolone acetonide, not otherw...",1114039732
680,J3301,"Injection, triamcinolone acetonide, not otherw...",1518372309


## Detect Common Patterns for Drug Prescription in the state of Florida

In [15]:
rollback_before_flag = True
rollback_after_flag = True

query = """

with report as (
select 
r.provider_npi,
r.code,
r.total_amount_paid,
sum(r.total_amount_paid) over (partition by r.code) as total_amount
from services_rendered r
     inner join (
         select provider_npi
            from providers
            where provider_state = 'FL') k on k.provider_npi = r.provider_npi
)

select distinct  d.drug_code, d.drug_description, count(distinct r.provider_npi) as provider_count, r.total_amount, r.total_amount / count(distinct r.provider_npi) as avg_amount
from drugs d
    join
    report as r on r.code = d.drug_code
group by d.drug_code, r.total_amount
order by provider_count desc

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,drug_code,drug_description,provider_count,total_amount,avg_amount
0,J3301,"Injection, triamcinolone acetonide, not otherw...",391,442.61,1.131995
1,J7308,Aminolevulinic acid hcl for topical administra...,173,66909.31,386.759017
2,J7345,Aminolevulinic acid hcl for topical administra...,46,71.84,1.561739
3,J9190,"Injection, fluorouracil, 500 mg",37,77.55,2.095946
4,Q4186,"Epifix, per square centimeter",9,1386.26,154.028889
5,J3245,"Injection, tildrakizumab, 1 mg",6,813.17,135.528333
6,J9260,"Methotrexate sodium, 50 mg",5,11.09,2.218000
7,Q4159,"Affinity, per square centimeter",5,2553.38,510.676000
8,J0702,"Injection, betamethasone acetate 3 mg and beta...",3,20.01,6.670000
9,J1100,"Injection, dexamethasone sodium phosphate, 1 mg",2,0.24,0.120000


## Building procedure table


In [16]:
connection.rollback()

query = """

drop table if exists procedures;

"""

cursor.execute(query)

connection.commit()

In [17]:
connection.rollback()


query = """

create table procedures (
  procedure_code varchar(32),
  procedure_description text,
  if_drug boolean,
  primary key (procedure_code)
);

"""

cursor.execute(query)

connection.commit()

In [18]:
connection.rollback()

query = f"""

copy procedures
from '{DATA_DIR}/procedures.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [19]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
procedure_code,
procedure_description
from procedures
limit 10

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,procedure_code,procedure_description
0,11102,"Biopsy of related skin growth, first growth"
1,11103,"Biopsy of related skin growth, each additional..."
2,17000,"Destruction of precancer skin growth, 1 growth"
3,17003,"Destruction of precancer skin growth, 2-14 gro..."
4,17110,"Destruction of skin growth, 1-14 growths"
5,99203,"New patient office or other outpatient visit, ..."
6,99212,Established patient office or other outpatient...
7,99213,Established patient office or other outpatient...
8,99214,Established patient office or other outpatient...
9,11403,"Removal of noncancer skin growth of body, arms..."


## Detect Common Patterns for Procedures in the state of Florida

In [20]:
rollback_before_flag = True
rollback_after_flag = True

query = """

with report as (
select 
r.provider_npi,
r.code,
r.total_amount_paid,
sum(r.total_amount_paid) over (partition by r.code) as total_amount
from services_rendered r
     inner join (
         select provider_npi
            from providers
            where provider_state = 'FL') k on k.provider_npi = r.provider_npi
)

select 
procedure_code, procedure_description, count(distinct r.provider_npi) as provider_count, r.total_amount, r.total_amount / count(distinct r.provider_npi) as avg_amount
from procedures p
    join
    report as r on r.code = p.procedure_code
group by p.procedure_code, r.total_amount
order by provider_count desc
limit 10 

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,procedure_code,procedure_description,provider_count,total_amount,avg_amount
0,99213,Established patient office or other outpatient...,964,88857.73,92.176068
1,17000,"Destruction of precancer skin growth, 1 growth",958,49690.60,51.869102
2,11102,"Biopsy of related skin growth, first growth",940,86332.59,91.843181
3,17003,"Destruction of precancer skin growth, 2-14 gro...",929,6180.13,6.652454
4,99203,"New patient office or other outpatient visit, ...",856,98542.11,115.119287
5,17110,"Destruction of skin growth, 1-14 growths",851,93515.20,109.888602
6,11103,"Biopsy of related skin growth, each additional...",832,43021.18,51.708149
7,99214,Established patient office or other outpatient...,823,106760.15,129.720717
8,99212,Established patient office or other outpatient...,738,42447.51,57.516951
9,17004,"Destruction of precancer skin growth, 15 or mo...",605,103047.20,170.325950


## Building Services table

In [21]:
connection.rollback()

query = """

drop table if exists services_rendered;

"""

cursor.execute(query)

connection.commit()

In [22]:
connection.rollback()


query = """

create table services_rendered (
  provider_npi varchar(32),
  code varchar(32),
  drug_indicator varchar(32),
  total_beneficiaries integer,
  total_services float,
  total_amount_paid float,
  primary key (provider_npi,code)
);

"""

cursor.execute(query)

connection.commit()

In [23]:
connection.rollback()

query = f"""

copy services_rendered
from '{DATA_DIR}/services_rendered.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [24]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
r.provider_npi as from_provider,
r.code as to_drug,
r.total_amount_paid as weight
from services_rendered r
     inner join (
         select provider_npi
            from providers
            where provider_state = 'FL') k on k.provider_npi = r.provider_npi
where r.drug_indicator = 'Y'
order by weight desc
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,from_provider,to_drug,weight
0,1821191099,Q4159,519.64
1,1871696021,Q4159,514.10
2,1144215898,Q4159,513.68
3,1831400100,Q4159,510.58
4,1831559665,Q4159,495.38
...,...,...,...
677,1386695054,J3301,0.97
678,1952369779,J0696,0.48
679,1407820764,J9250,0.22
680,1790892339,J1100,0.12


In [25]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select 
r.provider_npi as from_provider,
r.code as drug,
r.total_beneficiaries,
r.total_services,
r.total_amount_paid
from services_rendered r
     inner join (
         select provider_npi
            from providers
            where provider_state = 'CA') k on k.provider_npi = r.provider_npi
where r.drug_indicator = 'Y'
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,from_provider,drug,total_beneficiaries,total_services,total_amount_paid
0,1003043340,J7308,16,27,388.60
1,1003106014,J3301,37,79,1.14
2,1003106014,J7308,40,45,389.95
3,1003133216,J7345,13,21,1.58
4,1003158544,J3301,33,467,1.16
...,...,...,...,...,...
794,1992774681,J3301,20,34,1.04
795,1992774681,J7308,16,20,387.17
796,1992821797,J3301,12,30,1.13
797,1992852131,J7308,35,80,388.45
